In [16]:
from collections import defaultdict

from open_pref_eval.datasets.genies import GENIES, dist2datasets, GENIES_ALL, GENIES_TEST

In [52]:
import json
from pathlib import Path
import pandas as pd

all_dists = []
root = Path('distribution_shifts')
files = list(root.glob('**/*.json'))[3:]
for f in files:
    category, label = str(f.relative_to(root).parent), f.relative_to(root).stem
    with open(f) as json_file:
        data = json.load(json_file)
        for x in data:
            x['category'] = category
            x['label'] = label
            all_dists.append(x)
df = pd.DataFrame(all_dists).drop(columns=['target_reference'])
df

,source,target,category,label
0,pursue_goals,relinquish_power,probing,spurious_cues
1,creative_writing,biology_with_literary_style,probing,spurious_cues
2,alpaca_short,alpaca_long,probing,spurious_cues
3,alpaca_chat,illegal_dont_help,probing,spurious_cues
4,alpaca_mmlu,wrong_arc,probing,spurious_cues
...,...,...,...,...
120,change_my_view,cooking,extreme,skill
121,raven_matrices,us_history,extreme,skill
122,raven_matrices,code,extreme,skill
123,us_history,math,extreme,skill


In [72]:
df = pd.DataFrame(GENIES+GENIES_ALL+GENIES_TEST+all_dists).drop(columns=['target_reference'])
print(len(df))
df.drop_duplicates(inplace=True)
len(df)
df

211


,source,target,label,category
0,us_history_textbook,us_history_fiction,extreme,context
1,code_easy,code_hard,extreme,difficulty
2,alpaca_mmlu,spanish_output,extreme,encoding
3,math,change_my_view,extreme,skill
4,raven_matrices,us_history,extreme,skill
...,...,...,...,...
206,change_my_view,cooking,skill,extreme
207,raven_matrices,us_history,skill,extreme
208,raven_matrices,code,skill,extreme
209,us_history,math,skill,extreme


In [25]:
def get_all_test(GENIES):
    

    train2test = defaultdict(dict)

    for x in GENIES:
        s = x['source']
        # TODO instead of adding as target we should all kinds of OOD by tag
        l = x.get('category', 'oos')
        l += '|' + x.get('label', '')
        if l not in train2test[s]:
            train2test[s][l] = []
        train2test[s][l].append(
            x['target']
        )


    # now pick oos ones
    GALL = []
    for x in GENIES_ALL:
        GALL.append(x['source'])
        GALL.append(x['target'])

    # for x in GENIES:


    return train2test

print(get_all_test(GENIES))
print(get_all_test(GENIES_TEST))
print(get_all_test(GENIES_ALL))

defaultdict(<class 'dict'>, {'us_history_textbook': {'context|extreme': ['us_history_fiction']}, 'code_easy': {'difficulty|extreme': ['code_hard']}, 'alpaca_mmlu': {'encoding|extreme': ['spanish_output'], 'pretraining_similarity|extreme': ['raven_matrices', 'ranking_logic'], 'spurious_cues|probing': ['wrong_arc'], 'unwanted_personas|probing': ['truthful_qa', 'sycophancy_mimicry', 'survival_influence', 'reward_seeking']}, 'math': {'skill|extreme': ['change_my_view']}, 'raven_matrices': {'skill|extreme': ['us_history']}, 'alpaca_easy': {'difficulty|extreme': ['alpaca_hard']}, 'alpaca_low_quality': {'quality|extreme': ['alpaca_high_quality']}, 'alpaca_short': {'spurious_cues|extreme': ['alpaca_long']}, 'unhelpful_alpaca': {'oos|': ['illegal_dont_help']}})
defaultdict(<class 'dict'>, {'us_history_textbook': {'context|extreme': ['us_history_fiction']}})
defaultdict(<class 'dict'>, {'us_history_textbook': {'oos|': ['us_history_fiction']}, 'cooking': {'oos|': ['math', 'raven_matrices']}, 'us_